## Make dataset

In [ ]:
import string

def clean_text(corpus):
    # Remove punctuations from the corpus
    translator = str.maketrans('', '', string.punctuation)
    corpus = corpus.translate(translator)

    # Remove digits from the corpus
    remove_digits = str.maketrans('', '', string.digits)
    corpus = corpus.translate(remove_digits)
    return corpus

In [ ]:
def log_progress(sequence, every=10):
    from ipywidgets import IntProgress
    from IPython.display import display

    progress = IntProgress(min=0, max=len(sequence), value=0)
    display(progress)
    
    for index, record in enumerate(sequence):
        if index % every == 0:
            progress.value = index
        yield record

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

# Установите punkt, если еще не установлено
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')

def is_sentence_nltk(text):
    # Токенизируем текст на предложения
    sentences = sent_tokenize(text)
    # Проверяем, является ли исходная строка единственным предложением
    if len(sentences) != 1 or text.strip() != sentences[0]:
        return False
    # Проверяем количество слов
    words = word_tokenize(text)
    # Фильтруем только слова (убираем знаки препинания)
    words = [word for word in words if word.isalnum()]
    if len(words) < 2:
        return False
    
    return True


def get_first_two_words(text):
    # Превращаем несколько пробелов в один и убираем лишние пробелы
    text = ' '.join(text.split())
    
    # Токенизируем строку на слова
    words = word_tokenize(text)
    
    # Фильтруем только слова (убираем знаки препинания)
    filtered_words = [word for word in words if word.isalnum()]
    
    # Проверяем, есть ли хотя бы 2 слова
    if len(filtered_words) < 2:
        return "Недостаточно слов"
    
    # Возвращаем первые 2 слова
    return filtered_words[:2]

# Примеры
# print(is_sentence_nltk("I would like to receive the version of Newsletter"))  # True
# print(is_sentence_nltk("！！"))
# print(is_sentence_nltk("Это предложение. А это другое."))  # False
# print(is_sentence_nltk("Mr. Smith lives here."))  # True

In [ ]:


import json
import requests
import glob
import string
files = glob.glob("../data/path/steps/*")
files_txt = glob.glob("../data/path/txt/*")

import re  
def remove_all_cjk(text):
    return re.sub(r'[\u3000-\u30FF\u4E00-\u9FFF\uAC00-\uD7AF]', '', text)

def remove_punctuation(text):
    return re.sub(r'[:;!?“”]', '', text)

with open('../data/path/dataset.csv', 'w') as out_file:
    out_file.write("id,cluster,title,text\n")                
    for file in files:
        contents = open(file).readlines()
        step = int(contents[0].strip())
        text_path = f'../data/path/txt/{str(step).zfill(8)}.txt'                           
        try:
            text = open(text_path).read()
            text = remove_punctuation(remove_all_cjk(text))
            text = text.strip().replace("\n","").replace("\r","").replace("\t"," ").strip()
            text = re.sub(r'\s+', ' ', text.strip())
            text = clean_text(text)
            if text!="" and text!="For instantly purchase Please contact us to get this domain":
                if is_sentence_nltk(text):
                    title = " ".join(get_first_two_words(text))
                    out = [str(step), "0", title, "\"" + str(text)+ "\""] 
                    out_file.write(','.join(out)+"\n")
                    
                    if step > 50000:
                        break
        except FileNotFoundError:
            ...

        


In [ ]:
import pandas as pd
df = pd.read_csv("../data/path/dataset.csv", on_bad_lines='skip')
df.head()

In [ ]:
labels = df["title"].tolist()
labels[:10]

In [ ]:
documents = df['text'].tolist()
corpus = " ".join(documents).lower()

In [ ]:
corpus[:1000]

# Document Clustering

In [ ]:
cleaned_documents = []
for document in documents:
    document = clean_text(document).lower()
    cleaned_documents.append(document)
documents = cleaned_documents
documents[0]

In [ ]:
# !pip uninstall -y scikit-learn
# !pip install scikit-learn

# !pip uninstall -y scipy
# !pip install scipy

In [ ]:
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import time
from sklearn.cluster import KMeans

In [ ]:
class DocumentCluster:
    """Class to cluster document"""
    def __init__(self,model,data,labels):
        self.model = model
        self.data = data
        self.labels = labels
        self.model_name = self._get_model_name()
        
        self.cluster_pipeline = None
        
    
    def _get_model_name(self):
        """Get name of model being used"""
        model_name = str(self.model).split('(')[0]
        return model_name
            

    
    def train_model(self,ngram,use_idf):
        """ 
            Train model
            ngram: int
            use_idf: bool
            analyzer: string
        
        """
        current_time = time.time()
        self._ngram = ngram
        self._use_idf = use_idf
        
        # Set a pipeline
        # 1. The first step of the pipeline is to find count vectorizer
        # Countvectorizer converts a collection of text documents to a matrix of token counts
        # This implementation produces a sparse representation of the counts
        # 2. We then use TfidfTransformr
        # TfidfTransformer transforms a count matrix to a normalized tf or tf-idf representation
        # Tf means term-frequency while tf-idf means term-frequency times inverse document-frequency

        self.cluster_pipeline = Pipeline([('vect', CountVectorizer(ngram_range=(1,ngram), 
                analyzer="word")), ('tfidf', TfidfTransformer(use_idf=use_idf)), 
                ('model', self.model)])
        self._cluster = self.cluster_pipeline.fit(self.data)
        
        self._train_time = time.time() - current_time
        
        
        print("Training completed")
        print("Training time for {} : {} secs".format(self.model_name, self._train_time))
        
    
    def predict(self):
        """
        Predict cluster id for all clusters
        """
        self.label_id_dict = {}
        self.document_id_dict = {}
        self.cluster_id = []
        for i, document in enumerate(self.data):
            cluster_id = self.cluster_pipeline.predict([document])[0]
            self.label_id_dict.update({self.labels[i]:cluster_id})
            self.document_id_dict.update({document:cluster_id})
            self.cluster_id.append(cluster_id)
        
        return self.label_id_dict, self.document_id_dict

In [ ]:
cluster_number = 5

In [ ]:
model = KMeans(n_clusters=cluster_number, init = "k-means++", max_iter=150, n_init=1)

In [ ]:
doc_cluster = DocumentCluster(data=documents,model=model,labels=labels)

In [ ]:
doc_cluster.train_model(ngram=2, use_idf=True)

In [ ]:
result_label, result_docs = doc_cluster.predict()

In [ ]:
result_label

In [ ]:
#result_docs

In [ ]:
import pandas as pd
dataset_path = '../data/path/dataset.csv'
df = pd.read_csv(dataset_path, on_bad_lines='skip')
df.head()

# Add cluster data to dataset

In [ ]:
import pandas as pd
dataset_path = '../data/path/dataset.csv'
df = pd.read_csv(dataset_path, on_bad_lines='skip')
df.head()

for item in result_docs:
    search_string = item
    df.loc[df['text'].str.lower() == search_string, 'cluster'] = result_docs[item]
        
# Сохраняем обновленный DataFrame
df.to_csv(dataset_path, index=False) 
